In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from IPython.display import Video
import cv2

In [2]:
debug = True
if debug:
    epochs = 3
else:
    epochs = 100

err_tol = {
    'challenge': [ 0.30, 0.40, 0.50, 0.60, 0.70 ],
    'play': [ 0.15, 0.20, 0.25, 0.30, 0.35 ],
    'throwin': [ 0.15, 0.20, 0.25, 0.30, 0.35 ]
}
video_id_split = {
    'val':[
         '3c993bd2_0',
         '3c993bd2_1',
    ],
    'train':[
         '1606b0e6_0',
         '1606b0e6_1',
         '35bd9041_0',
         '35bd9041_1',
         '407c5a9e_1',
         '4ffd5986_0',
         '9a97dae4_1',
         'cfbe2e94_0',
         'cfbe2e94_1',
         'ecf251d4_0',
    ]
}
event_names = ['challenge', 'throwin', 'play']

# load label data

Label is asigned as follows:
<pre>
  ・
  ・
  ・
start
  ・                   -> backgraund
event1 - tolerances
  ・                   -> event1
event1 + tolerances
  ・                   -> backgraund
event2 - tolerances
  ・                   -> event2
event2 + tolerances
  ・                   -> backgraund
end
  ・                   -> not used
start
  ・
  ・
  ・
</pre>


In [3]:
df = pd.read_csv("../input/dfl-bundesliga-data-shootout/train.csv")
additional_events = []
for arr in df.sort_values(['video_id','time','event','event_attributes']).values:
    if arr[2] in err_tol:
        tol = err_tol[arr[2]][0]/2
        additional_events.append([arr[0], arr[1]-tol, 'start_'+arr[2], arr[3]])
        additional_events.append([arr[0], arr[1]+tol, 'end_'+arr[2], arr[3]])
df = pd.concat([df, pd.DataFrame(additional_events, columns=df.columns)])
df = df[~df['event'].isin(event_names)]
df = df.sort_values(['video_id', 'time'])
df

,video_id,time,event,event_attributes
0,1606b0e6_0,200.265822,start,NaN
0,1606b0e6_0,201.000000,start_challenge,['ball_action_forced']
1,1606b0e6_0,201.300000,end_challenge,['ball_action_forced']
2,1606b0e6_0,202.765822,end,NaN
3,1606b0e6_0,210.124111,start,NaN
...,...,...,...,...
11214,ecf251d4_0,3058.072895,end,NaN
11215,ecf251d4_0,3068.280519,start,NaN
8762,ecf251d4_0,3069.472000,start_throwin,['pass']
8763,ecf251d4_0,3069.622000,end_throwin,['pass']


In [4]:
def extract_training_images(args):
    video_id, split = args
    video_path = f"../input/dfl-bundesliga-data-shootout/train/{video_id}.mp4"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return
    fps = cap.get(cv2.CAP_PROP_FPS)
    time_interval = 1/fps

    df_video = df[df.video_id == video_id]
    if debug:
        df_video = df_video.head(20)
    print(split, video_id, df_video.shape)

    #crr_statu => background, play, challenge, throwin
    arr = df_video[['time','event']].values
    for idx in range(len(arr)-1):
        crr_time = arr[idx,0]
        nxt_time = arr[idx+1,0]
        crr_event = arr[idx,1]

        crr_event = crr_event
        if crr_event == 'start':
            crr_status = 'background'
        elif crr_event == 'end':
            # should use as background?
            continue
        else:
            start_or_end, crr_status = crr_event.split('_', 1)
            if start_or_end == 'end':
                crr_status = 'background'

        result_dir = f"./work/split_images/{split}/{crr_status}"
        if not os.path.exists(result_dir):
            os.makedirs(result_dir, exist_ok=True)

        this_time = crr_time
        while this_time < nxt_time:
            frame_num = int(this_time*fps)

            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()
            out_file = f'{result_dir}/{video_id}_{frame_num:06}.jpg'
            cv2.imwrite(out_file, frame)

            if crr_status == 'background':
                this_time += time_interval*10
            else:
                this_time += time_interval
      
!rm -rf ./work/split_images/
for split in video_id_split:
    video_ids = video_id_split[split]
    for video_id in video_ids:            
        extract_training_images([video_id, split])
print('done')

val 3c993bd2_0 (20, 4)
val 3c993bd2_1 (20, 4)
train 1606b0e6_0 (20, 4)
train 1606b0e6_1 (20, 4)
train 35bd9041_0 (20, 4)
train 35bd9041_1 (20, 4)
train 407c5a9e_1 (20, 4)
train 4ffd5986_0 (20, 4)
train 9a97dae4_1 (20, 4)
train cfbe2e94_0 (20, 4)
train cfbe2e94_1 (20, 4)
train ecf251d4_0 (20, 4)
done


In [5]:
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import os
import sys
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

%pylab inline

import cv2
from PIL import Image

import torch
torch.manual_seed(0)  # 减少随机性
torch.backends.cudnn.deterministic = False  # 是否有确定性
torch.backends.cudnn.benchmark = True  # 自动寻找最适合当前配置的高效算法，提高运行效率

class DFLDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])
        img = img.astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            img = self.transform(image=img)['image']
        return img, torch.from_numpy(np.array(self.img_label[index]))

    def __len__(self):
        return len(self.img_path)

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['split']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
train_path = glob.glob('./work/split_images/train/*/*')
train_label = [x.split('/')[-2] for x in train_path]

val_path = glob.glob('./work/split_images/val/*/*')
val_label = [x.split('/')[-2] for x in val_path]

train_df = pd.DataFrame({
    'path': train_path,
    'label': train_label
})
train_df['label_int'], lbl = pd.factorize(train_df['label'])
lbl = list(lbl)
train_df = train_df.sample(frac=1.0)

val_df = pd.DataFrame({
    'path': val_path,
    'label': val_label
})
val_df['label_int'] = val_df['label'].apply(lambda x: lbl.index(x))
# train_df = train_df.sample(frac=1.0)

model = models.resnet18(True)
model.fc = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(in_features=512, out_features=4, bias=True)
)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    return train_loss/len(train_loader)


def validate(val_loader, model, criterion):
    model.eval()

    val_acc = 0.0

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            val_acc += (output.argmax(1) == target).sum().item()

    return val_acc / len(val_loader.dataset)


def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0

    test_pred = []
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)


# 随机拆分
train_loader = torch.utils.data.DataLoader(
    DFLDataset(train_df['path'].values, train_df['label_int'].values,
                  A.Compose([
                      A.Resize(300, 300),
                      A.RandomContrast(p=0.5),
                      A.RandomBrightness(p=0.5),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                      ])

                  ), batch_size=10, shuffle=True, num_workers=4, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    DFLDataset(val_df['path'].values, val_df['label_int'].values,
                  A.Compose([
                      A.Resize(300, 300),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                      ])
                  ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)


model = model.to('cuda')
criterion = nn.CrossEntropyLoss().cuda()  # 自带softmax
optimizer = torch.optim.SGD(model.parameters(), 0.005)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=5, mode="triangular2")

best_acc = 0
for _ in range(5):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc = validate(val_loader, model, criterion)

    if val_acc > best_acc:
        torch.save(model.state_dict(), 'model.pth')
        best_acc = val_acc

    scheduler.step()
    print(train_loss, val_acc)

/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1641: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1615: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1.2115342645875868 0.6428571428571429
1.4764612807381539 0.6428571428571429
1.2990599668795062 0.6428571428571429
1.2188447740289472 0.3482142857142857
1.2857114893774833 0.6428571428571429


In [8]:
!rm -rf ./work/split_images/